In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, ReLU as KerasReLU, BatchNormalization, Dropout, InputLayer, GlobalAveragePooling2D, Lambda, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy as KerasSCCE
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-05-30 19:54:40.340331: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-30 19:54:40.831519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748609680.953541  395946 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748609680.987175  395946 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748609681.446045  395946 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
import sys

sys.path.append('../../')

from src.cnn.cnn import CNN
from src.cnn.layers import Conv2DLayer, ReLULayer, PoolingLayer, FlattenLayer, DenseLayer, SoftmaxLayer
from src.cnn.losses import SparseCategoricalCrossentropy

In [4]:
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()

In [5]:
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [6]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full, test_size=0.2, random_state=42
)

In [7]:
num_classes = 10
input_shape = x_train.shape[1:]

In [8]:
y_train_sparse = y_train.flatten()
y_val_sparse = y_val.flatten()
y_test_sparse = y_test.flatten()

In [9]:
print(f"x_train: {x_train.shape}, y_train_sparse: {y_train_sparse.shape}")
print(f"x_val: {x_val.shape}, y_val_sparse: {y_val_sparse.shape}")
print(f"x_test: {x_test.shape}, y_test_sparse: {y_test_sparse.shape}")

x_train: (40000, 32, 32, 3), y_train_sparse: (40000,)
x_val: (10000, 32, 32, 3), y_val_sparse: (10000,)
x_test: (10000, 32, 32, 3), y_test_sparse: (10000,)


In [10]:
base_conv_params = [
    {'filters': 32, 'kernel_size': (3,3)},
    {'filters': 32, 'kernel_size': (3,3), 'pool': 'max'},
    {'filters': 64, 'kernel_size': (3,3)},
    {'filters': 64, 'kernel_size': (3,3), 'pool': 'max'},
    {'filters': 128, 'kernel_size': (3,3)},
    {'filters': 128, 'kernel_size': (3,3), 'pool': 'max'},
]

base_dense_units = [521]

In [11]:
keras_cnn_model = tf.keras.models.load_model('keras_cnn_model.keras')

I0000 00:00:1748609713.251947  395946 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [12]:
for i, layer in enumerate(keras_cnn_model.layers):
    if layer.get_weights():
        weights = layer.get_weights()
        print(f"Layer {i}: {layer.name}")
        for j, w in enumerate(weights):
            print(f"  Param {j} wieight: {w}")
    else:
        print(f"Layer {i}: {layer.name} (No weights)")

Layer 0: conv2d
  Param 0 wieight: [[[[ 2.93029640e-02 -1.95078269e-01  2.98273861e-02 -1.53873324e-01
    -2.10194483e-01  1.98063686e-01 -2.23256066e-01 -2.20748410e-02
     1.65085018e-01 -4.90335189e-02  1.59493595e-01  3.36729810e-02
    -1.85174942e-01 -7.78291598e-02  8.39786902e-02 -6.30947277e-02
    -7.47838542e-02  2.80072670e-02 -7.57440180e-02  1.04095362e-01
     7.34699443e-02  1.97189540e-01 -1.71759099e-01 -1.63141072e-01
    -1.98106289e-01 -6.26172200e-02 -1.42005131e-01 -1.02393411e-01
    -2.87740171e-01  3.08591616e-03 -1.34696469e-01  5.32340072e-02]
   [ 4.62646149e-02 -5.68173118e-02 -1.59721226e-01 -3.16007845e-02
    -1.87986046e-01  2.64558531e-02  1.65793076e-02 -1.21763311e-01
     6.87169358e-02  6.41492903e-02  5.53651713e-02  1.23144053e-01
    -1.07268542e-01 -3.83552089e-02  1.16693027e-01  6.65405244e-02
    -2.62150496e-01 -2.60094460e-03  6.20519230e-03 -8.84957984e-02
     5.08796610e-02  1.45600230e-01 -1.67295381e-01  1.00275397e-01
    -4.35686

In [13]:
manual_cnn_model = CNN()

for params in base_conv_params:
    manual_cnn_model.add_layer(Conv2DLayer(num_filters=params['filters'], filter_size=params['kernel_size'], padding='same'))
    manual_cnn_model.add_layer(ReLULayer())
    
    pool_type = params.get('pool')
    if pool_type == 'max':
        manual_cnn_model.add_layer(PoolingLayer(pool_size=(2,2), stride=2, mode='max'))
    elif pool_type == 'average':
        manual_cnn_model.add_layer(PoolingLayer(pool_size=(2,2), stride=2, mode='average'))
        
manual_cnn_model.add_layer(FlattenLayer())

for units in base_dense_units:
    manual_cnn_model.add_layer(DenseLayer(output_dim=units))
    manual_cnn_model.add_layer(ReLULayer())

manual_cnn_model.add_layer(DenseLayer(output_dim=num_classes))

manual_cnn_model.load_weights_from_keras(keras_cnn_model)

manual_cnn_model.loss_function = SparseCategoricalCrossentropy(from_logits=True)

Attempting to load weights from Keras model...
Comparing Manual: Conv2DLayer with Keras: Conv2D
  Matched and processed manual: Conv2DLayer with Keras: Conv2D
Comparing Manual: ReLULayer with Keras: ReLU
  Matched Keras ReLU type for manual ReLULayer.
  Matched and processed manual: ReLULayer with Keras: ReLU
Comparing Manual: Conv2DLayer with Keras: Conv2D
  Matched and processed manual: Conv2DLayer with Keras: Conv2D
Comparing Manual: ReLULayer with Keras: ReLU
  Matched Keras ReLU type for manual ReLULayer.
  Matched and processed manual: ReLULayer with Keras: ReLU
Comparing Manual: PoolingLayer with Keras: MaxPooling2D
  Matched and processed manual: PoolingLayer with Keras: MaxPooling2D
Comparing Manual: Conv2DLayer with Keras: Conv2D
  Matched and processed manual: Conv2DLayer with Keras: Conv2D
Comparing Manual: ReLULayer with Keras: ReLU
  Matched Keras ReLU type for manual ReLULayer.
  Matched and processed manual: ReLULayer with Keras: ReLU
Comparing Manual: Conv2DLayer with 

In [ ]:
x_test_subset_verify = x_test[:200]
y_test_subset_verify_sparse = y_test_sparse[:200]

keras_subset_logits_verify = keras_cnn_model.predict(x_test_subset_verify)
keras_subset_proba_verify = tf.nn.softmax(keras_subset_logits_verify).numpy()
keras_subset_preds_verify = np.argmax(keras_subset_proba_verify, axis=1)
f1_keras_verify = f1_score(y_test_subset_verify_sparse, keras_subset_preds_verify, average='macro')
acc_keras_verify = accuracy_score(y_test_subset_verify_sparse, keras_subset_preds_verify)
print(f"Keras (on subset {len(x_test_subset_verify)}) - Accuracy: {acc_keras_verify:.4f}, Macro F1: {f1_keras_verify:.4f}")

time_start = time.time()
manual_subset_proba_verify = manual_cnn_model.predict_proba(x_test_subset_verify)
manual_subset_preds_verify = np.argmax(manual_subset_proba_verify, axis=1)
time_end = time.time()
print(f"Manual Model Prediction Time for Subset: {time_end - time_start:.4f} seconds")

\

print("\nRaw probability Comparison")
for i in range(5):
    print(f"Sample {i}:")
    print(f"  Keras Probs (first 5):   {keras_subset_proba_verify[i, :5]}")
    print(f"  Manual Probs (first 5): {manual_subset_proba_verify[i, :5]}")
    diff = np.sum(np.abs(keras_subset_proba_verify[i] - manual_subset_proba_verify[i]))
    print(f"  Sum of Absolute Differences in Probs: {diff:.6e}")
    if diff > 1e-2:
            print(f"  INFO: Probabilities differ for sample {i}, potentially due to BN/Dropout behavior in Keras inference vs scratch.")


I0000 00:00:1748609717.764008  396128 service.cc:152] XLA service 0x7f7310015280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748609717.764099  396128 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-05-30 19:55:17.849778: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748609717.976407  396128 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-30 19:55:19.867673: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_93', 52 bytes spill stores, 52 bytes spill loads

2025-05-30 19:55:20.851773: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_

1/7 ━━━━━━━━━━━━━━━━━━━━ 52s 9s/step

I0000 00:00:1748609726.140348  396128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-05-30 19:55:27.435644: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_93', 36 bytes spill stores, 36 bytes spill loads

2025-05-30 19:55:28.071744: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_93', 36 bytes spill stores, 36 bytes spill loads

2025-05-30 19:55:28.287971: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_93', 1460 bytes spill stores, 1460 bytes spill loads



7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 532ms/step
Keras (on subset 200) - Accuracy: 0.8650, Macro F1: 0.8601
Manual Model Prediction Time for Subset: 219.0255 seconds
Manual (on subset 200, loaded weights) - Accuracy: 0.8650, Macro F1: 0.8601

Raw probability Comparison
Sample 0:
  Keras Probs (first 5):   [7.2144394e-07 4.5642827e-04 1.3333153e-04 9.9538112e-01 9.9156502e-08]
  Manual Probs (first 5): [7.14222900e-07 4.54180847e-04 1.32367074e-04 9.95406931e-01
 9.78562523e-08]
  Sum of Absolute Differences in Probs: 5.161866e-05
Sample 1:
  Keras Probs (first 5):   [4.7253216e-06 6.2609273e-03 2.4768746e-12 5.7769127e-09 1.1951123e-11]
  Manual Probs (first 5): [4.67968069e-06 6.26356242e-03 2.42920455e-12 5.69989310e-09
 1.16998940e-11]
  Sum of Absolute Differences in Probs: 5.202723e-06
Sample 2:
  Keras Probs (first 5):   [1.3935267e-05 1.2774125e-04 2.9706320e-09 4.1362412e-07 2.6599283e-09]
  Manual Probs (first 5): [1.38236711e-05 1.27199977e-04 2.93347680e-09 4.10538624e-07
 2.62288303

: 

In [ ]:
time_start_10k = time.time()
manual_proba = manual_cnn_model.predict_proba(x_test)
manual_preds = np.argmax(manual_proba, axis=1)
time_end_10k = time.time()
print(f"Manual Model Prediction Time for Subset: {time_end_10k - time_start_10k:.4f} seconds")

f1_manual = f1_score(y_test_sparse, manual_preds, average='macro')
acc_manual = accuracy_score(y_test_sparse, manual_preds)
print(f"Manual (on subset {len(x_test)}, loaded weights) - Accuracy: {acc_manual:.4f}, Macro F1: {f1_manual:.4f}")